In [ ]:
# pip install gspread oauth2client

In [36]:
import gspread
import os
from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd
import numpy as np
from datetime import datetime

In [37]:
import csv
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

In [38]:
# ## Google Sheets API Authentication
def authenticate():
    creds = None
    # scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
    scope = ["https://www.googleapis.com/auth/spreadsheets", 
             "https://www.googleapis.com/auth/drive"]
    # Check if token.json file exists for previously saved credentials
    if os.path.exists("token.json"):
        creds = Credentials.from_authorized_user_file("token.json", scope)
    
    # If no valid credentials, initiate login process
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file("credentials.json", scope)
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open("token.json", "w") as token:
            token.write(creds.to_json())
    client = gspread.authorize(creds)
    
    return client

In [39]:
# ## Google Sheets API Authentication
# def authenticate():
#     # Define the scope of the API
#     scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
    
#     # Provide the credentials file path
#     creds = ServiceAccountCredentials.from_json_keyfile_name("credentials.json", scope)
    
#     # Authorize the client to interact with the Google Sheets API
#     client = gspread.authorize(creds)
    
#     return client

In [40]:
# Handle data serialization, including date formats
def serialize_data(df):
    # Convert datetime-like columns to string format
    for col in df.columns:
        if pd.api.types.is_datetime64_any_dtype(df[col]):
            df[col] = df[col].apply(lambda x: x.strftime('%Y-%m-%d') if pd.notnull(x) else '')
    
    return df

In [69]:
# Replace data in the Google Sheet while preserving headers
def replace_data_in_sheet(spreadsheet_id, sheet_name, data_file):
    # Authenticate and open the spreadsheet
    client = authenticate()
    sheet = client.open_by_key(spreadsheet_id)
    worksheet = sheet.worksheet(sheet_name)
    
    # Read the existing data to get the headers
    headers = worksheet.row_values(1)  # Assuming headers are in the first row
    
    # Load new data from the CSV file
    # data_df = serialize_data(pd.read_csv(data_file))
    
    # Convert the dataframe to a list of lists for gspread
    new_data = data_df.values.tolist()

    # Clear the previous data in the sheet, but preserve the headers
    worksheet.clear()
    worksheet.append_row(headers)  # Add the headers back to the sheet
    batch_size=100
    
    # Update the sheet with the new data
    # Now use batch update to add data in chunks of 'batch_size'
    for i in range(0, len(new_data), batch_size):
        batch = new_data[i:i + batch_size]
        cell_range = f"A{i+2}"  # Skip the first row (headers)
        worksheet.update(cell_range, batch)
    
    print(f"Data from {data_file} has been replaced successfully.")

In [57]:
# # Main execution
# if __name__ == "__main__":
# Your Google Sheets spreadsheet ID and sheet name
SPREADSHEET_ID = "1P8oONrbhDqPdj7ubLUWCmfXJd6eOexO5FzWWgOvzIlE"
SHEET_NAME = "Sheet1"  # Change as needed
DATA_FILE = "data.csv"  # Path to your local data file

In [58]:
df = pd.read_csv('data.csv')

In [59]:
df.head()

,ShipmentId,TierType,Type,ReverseShipmentId,ForwardReverseType,TotalPrice,DeliveryPincode,Status,DeliveryHub,CurrentHub,...,NumberOfAttempts,CsNotes,HubNotes,OnHoldByOpsReason,OnHoldByOpsDate,BagId,BagStatus,ConsignmentId,OrderId,ItemDescription
0,FMPP1819610045,REGULAR,PP,NaN,NORMAL_FORWARD,498,421305,Delivered,BulkHub_BHI,BulkHub_BHI,...,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,OD428913123487126100,MITHALI HotPot chapati box/chapati case Casser...
1,FMPC2972423738,REGULAR,COD,NaN,NORMAL_FORWARD,12290,421601,Delivered,BulkHub_BHI,BulkHub_BHI,...,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,OD428893490469098100,Godrej 8 kg Semi Automatic Top Load Washing Ma...
2,FMPC2970867951,ECONOMY,COD,NaN,NORMAL_FORWARD,1788,144004,Expected,satellitehub_JLD,BulkHub_BHI,...,0,Marked_As_RTO,NaN,NaN,NaN,NaN,NaN,NaN,OD328879078257415100,APTITUDE 30KG PVC Weight Dumbbell Set with 4 R...
3,FMPC2968732983,REGULAR,COD,NaN,NORMAL_FORWARD,934,700128,Delivered,Bulkhub_KOL10,Bulkhub_KOL10,...,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,OD328877335539322100,Tucker 3 In 1 Convertible Dumbbells Barbell H...
4,FMPC2968930927,REGULAR,COD,NaN,NORMAL_FORWARD,574,700125,Delivered,Bulkhub_KOL10,Bulkhub_KOL10,...,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,OD428858903532838100,APRAS Combo of Water Tank Y Type Screen Filter...


In [60]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 749 entries, 0 to 748
Data columns (total 27 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   ShipmentId            749 non-null    object 
 1   TierType              749 non-null    object 
 2   Type                  749 non-null    object 
 3   ReverseShipmentId     2 non-null      object 
 4   ForwardReverseType    749 non-null    object 
 5   TotalPrice            749 non-null    int64  
 6   DeliveryPincode       749 non-null    int64  
 7   Status                749 non-null    object 
 8   DeliveryHub           749 non-null    object 
 9   CurrentHub            749 non-null    object 
 10  CustomerPromiseDate   749 non-null    object 
 11  LogisticsPromiseDate  113 non-null    object 
 12  LatestUpdateTime      749 non-null    object 
 13  FirstReceiveTime      749 non-null    object 
 14  FirstReceiveHub       749 non-null    object 
 15  LastReceiveTime       7

In [61]:
df.isna().sum()

ShipmentId                0
TierType                  0
Type                      0
ReverseShipmentId       747
ForwardReverseType        0
TotalPrice                0
DeliveryPincode           0
Status                    0
DeliveryHub               0
CurrentHub                0
CustomerPromiseDate       0
LogisticsPromiseDate    636
LatestUpdateTime          0
FirstReceiveTime          0
FirstReceiveHub           0
LastReceiveTime           0
FirstAssignedHub          0
NumberOfAttempts          0
CsNotes                 504
HubNotes                702
OnHoldByOpsReason       749
OnHoldByOpsDate         749
BagId                   749
BagStatus               749
ConsignmentId           749
OrderId                   0
ItemDescription           0
dtype: int64

In [62]:
df.isin([np.inf, -np.inf, np.nan]).sum()

ShipmentId                0
TierType                  0
Type                      0
ReverseShipmentId       747
ForwardReverseType        0
TotalPrice                0
DeliveryPincode           0
Status                    0
DeliveryHub               0
CurrentHub                0
CustomerPromiseDate       0
LogisticsPromiseDate    636
LatestUpdateTime          0
FirstReceiveTime          0
FirstReceiveHub           0
LastReceiveTime           0
FirstAssignedHub          0
NumberOfAttempts          0
CsNotes                 504
HubNotes                702
OnHoldByOpsReason       749
OnHoldByOpsDate         749
BagId                   749
BagStatus               749
ConsignmentId           749
OrderId                   0
ItemDescription           0
dtype: int64

In [63]:
df.columns

Index(['ShipmentId', 'TierType', 'Type', 'ReverseShipmentId',
       'ForwardReverseType', 'TotalPrice', 'DeliveryPincode', 'Status',
       'DeliveryHub', 'CurrentHub', 'CustomerPromiseDate',
       'LogisticsPromiseDate', 'LatestUpdateTime', 'FirstReceiveTime',
       'FirstReceiveHub', 'LastReceiveTime', 'FirstAssignedHub',
       'NumberOfAttempts', 'CsNotes', 'HubNotes', 'OnHoldByOpsReason',
       'OnHoldByOpsDate', 'BagId', 'BagStatus', 'ConsignmentId', 'OrderId',
       'ItemDescription'],
      dtype='object')

In [64]:
# Check for NaN, Infinity, and -Infinity in the entire DataFrame
special_values = df.isin([np.inf, -np.inf, np.nan])

# Find which columns contain these special values
columns_with_special_values = special_values.any()

In [65]:
# Display columns with NaN, Infinity, or -Infinity
print("Columns with special float values:", df.columns[columns_with_special_values])

Columns with special float values: Index(['ReverseShipmentId', 'LogisticsPromiseDate', 'CsNotes', 'HubNotes',
       'OnHoldByOpsReason', 'OnHoldByOpsDate', 'BagId', 'BagStatus',
       'ConsignmentId'],
      dtype='object')


In [66]:
df.replace([np.inf, -np.inf, np.nan], 'null', inplace=True)

In [67]:
data_df = serialize_data(df)

In [70]:
replace_data_in_sheet(SPREADSHEET_ID, SHEET_NAME, data_df)

C:\Users\hp\AppData\Local\Temp\ipykernel_17952\2727272989.py:27: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  worksheet.update(cell_range, batch)


Data from          ShipmentId TierType Type ReverseShipmentId ForwardReverseType  \
0    FMPP1819610045  REGULAR   PP              null     NORMAL_FORWARD   
1    FMPC2972423738  REGULAR  COD              null     NORMAL_FORWARD   
2    FMPC2970867951  ECONOMY  COD              null     NORMAL_FORWARD   
3    FMPC2968732983  REGULAR  COD              null     NORMAL_FORWARD   
4    FMPC2968930927  REGULAR  COD              null     NORMAL_FORWARD   
..              ...      ...  ...               ...                ...   
744  FMPC2979471836  ECONOMY  COD              null     NORMAL_FORWARD   
745  FMPC2977934138  REGULAR  COD              null     NORMAL_FORWARD   
746  FMPC2972509436  REGULAR  COD              null     NORMAL_FORWARD   
747  FMPC2980940816  REGULAR  COD              null     NORMAL_FORWARD   
748  FMPP1815985790  ECONOMY   PP              null     NORMAL_FORWARD   

     TotalPrice  DeliveryPincode     Status        DeliveryHub  \
0           498           421305  D